In [50]:
%matplotlib inline

In [49]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display
import numpy as np


In [54]:
class Body(plt.Circle):
    def __init__(self, x,y,radius = 0.2,color='green'):
        self.x = x
        self.y = y
        self.body = plt.Circle((self.x,self.y), radius = radius,color = color)


circleA = Body(1,1)
circleB = Body(3,3, radius= 0.08,color='red')

fig = plt.figure(figsize= (5,5))
ax = fig.add_subplot(111)
ax.add_artist(circleA.body)
ax.add_artist(circleB.body)

lines = plt.plot([])
line = lines[0]


plt.xlim(0,10)
plt.ylim(0,10)

def animate(frame):
    circleA.x+=0.01
    circleA.y = circleA.x **2
    circleB.x += 0.02
    circleB.y += 0.02
    circleA.body.center = (circleA.x, circleA.y)
    circleB.body.center = (circleB.x, circleB.y)
    line.set_data((circleA.x,circleA.y))
    line.set_data((circleB.x, circleB.y))
    
    
anim = FuncAnimation(fig, animate,frames=200, interval=20)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()